# Saved Model

在使用 TensorFlow Serving 时，会用到这种格式的模型。该格式为 GraphDef 和 CheckPoint 的结合体。从 SavedModel 中可以提取 GraphDef 和 CheckPoint 对象。

模型的存储结构如下：

└── 1
···├── saved_model.pb
···└── variables
·········├── variables.data-00000-of-00001
·········└── variables.index

In [3]:
import tensorflow as tf

sess = tf.InteractiveSession()

w1 = tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2 = tf.constant([1, 2, 3, 4, 5], shape=[5] , name='w2')
print(w1)
print(w2)
# 使用tf.train.Saver()定义一个存储器对象
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# 使用saver.save保存模型
#saver.save(sess, 'model')

# 保存相应变量到指定文件, 如果指定 global_step, 则实际保存的名称变为 model.ckpt-xxxx
# saver.save(sess, "./model.ckpt", global_step=epoch)
saver.save(sess, './ckpts/model.ckpt', global_step=1)

Instructions for updating:
Colocations handled automatically by placer.
<tf.Variable 'w1:0' shape=(2,) dtype=float32_ref>
Tensor("w2:0", shape=(5,), dtype=int32)


'./ckpts/model.ckpt-1'

In [4]:
! rm -rf './saved_model'

## Saved Model 保存

In [5]:
saved_model_dir = './saved_model'
builder = tf.saved_model.builder.SavedModelBuilder(saved_model_dir)
builder.add_meta_graph_and_variables(sess, ['saved_model_demo'])
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./saved_model/saved_model.pb


b'./saved_model/saved_model.pb'

add_meta_graph_and_variables方法：

导入graph的信息以及变量，这个方法假设变量都已经初始化好了，对于每个SavedModelBuilder这个方法一定要执行一次用于导入第一个meta graph。

第一个参数：

传入当前的session，包含了graph的结构与所有变量。

第二个参数：

给当前需要保存的meta graph一个标签，标签名可以自定义，在之后载入模型的时候，需要根据这个标签名去查找对应的MetaGraphDef，找不到就会报找不到的错误。

标签也可以选用系统定义好的参数，如tf.saved_model.tag_constants.SERVING与tf.saved_model.tag_constants.TRAINING。

Save方法就是将模型序列化到指定目录底下。

保存好以后到saved_model_dir目录下，会有一个saved_model.pb文件以及variables文件夹。顾名思义，variables保存所有变量，saved_model.pb用于保存模型结构等信息。

### 模型的载入

使用tf.saved_model.loader.load方法就可以载入模型

In [8]:
meta_graph_def = tf.saved_model.loader.load(sess, ['saved_model_demo'], saved_model_dir)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./saved_model/variables/variables


In [12]:
w1 = sess.graph.get_tensor_by_name('w1:0')
print(sess.run(w1))

[-0.36471716 -0.9248953 ]
